[View in Colaboratory](https://colab.research.google.com/github/schwaaweb/aimlds1_08-UnsupervisedLearning/blob/master/M08_A--DJ--Clustering_Assignment.ipynb)

In this assignment, you will work with one of the UCI datasets: 
http://scikit-learn.org/stable/modules/classes.html#module-sklearn.datasets

### Objectives
 * Learn how to run clustering in python with different methods
 * Output and interpret clustering metrics
 * Experiment with PCA/data normalization of the dataset

On the example of wine, load the dataset and run the clustering algorithms on it.

---



In [1]:
from time import time
import numpy as np
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import load_wine
from sklearn.decomposition import PCA

wine_dataset = load_wine()
data = wine_dataset.data
print(data)
n_samples, n_features = data.shape
n_clusters = len(np.unique(wine_dataset.target))
labels = wine_dataset.target

print("n_digits: %d, \t n_samples %d, \t n_features %d"
      % (n_clusters, n_samples, n_features))

[[1.423e+01 1.710e+00 2.430e+00 ... 1.040e+00 3.920e+00 1.065e+03]
 [1.320e+01 1.780e+00 2.140e+00 ... 1.050e+00 3.400e+00 1.050e+03]
 [1.316e+01 2.360e+00 2.670e+00 ... 1.030e+00 3.170e+00 1.185e+03]
 ...
 [1.327e+01 4.280e+00 2.260e+00 ... 5.900e-01 1.560e+00 8.350e+02]
 [1.317e+01 2.590e+00 2.370e+00 ... 6.000e-01 1.620e+00 8.400e+02]
 [1.413e+01 4.100e+00 2.740e+00 ... 6.100e-01 1.600e+00 5.600e+02]]
n_digits: 3, 	 n_samples 178, 	 n_features 13
